# Differentiable Dynamic Programming
This section is to solve the Bellman equation using differentiable dynamic programming (DDP).
Refer to [Differentiable Optimal Control via Differential Dynamic Programming](https://arxiv.org/pdf/2209.01117.pdf) and a Python implementation [here](https://github.com/neka-nat/ddp-gym/blob/master/ddp_gym.py).

DDP is designed for solving nonlinear dynamic system optimal control, which performs a quadratic approximation of the cost based on Taylor Series.




## System Dynamics

\begin{align}
x_{t+1} &= f(x_t, u_t) \\
x_0 &= x(0)
\end{align}

## Cost Function

The `cost-to-go` function at time `t` is shown as follows:

\begin{align}
    J(x_t, U_t) &= \sum_{k=t}^{N-1}\ell(x_k, u_k) + \ell_f(x_N) \\
    U_t &= \{u_t, ..., u_{N-1}\}
\end{align}

Minimum `cost-to-go` or `value function` at time `t` is then:

\begin{align}
    V_t(x_t) &= \min_{U_t} J_t(x_t, U_t) \\
            &= \min_{u_t, ..., u_{N-1}} \sum_{k=t}^{N-1} \ell(x_k, u_k) + \ell_f(x_{N}) \\
            &= \min_{u_t} [\ell(x_t, u_t) + V_{t+1}(x_{t+1})] \\
            &= \min_{u_t} [\ell(x_t, u_t) + V_{t+1}(f(x_t, u_t)] \\
            &= \min_{u_t} Q_t(x_t, u_t)
\end{align}

## Second-order Approximation of Q-function
For linear system and quadratic cost function $\ell$, $Q$ is quadratic.
But when the system is nonlinear, $Q$ is no longer quadratic.
DDP is to approximate the action-value function $Q$ in a quadratic format, which can be done based on Taylor Series.

DDP iterates between
- (i) minimizing the quadratic approximation of the Q-function in a `backward pass`, and 
- (ii) integrating the system dynamics in a `forward pass`

Given a sequence of nominal control trajectory $U_t^r = \{u_t^r, ..., u_{N-1}^r\}$, and a sequence of nominal state trajectory $X_t^r = \{x_t^r, ..., x_N^r\}$, we can approximate $Q_t(u_t, x_t)$ around the given nominal conditions based on Taylor series. 
Let's drop the subscript `t` for brivity:

\begin{equation}
\hat Q(x_t,u_t) = Q(x_t^r, u_t^r) + Q_x \Delta x_t + Q_u \Delta u_t + \frac{1}{2}\Delta x_t^T Q_{xx} \Delta x_t + \frac{1}{2}\Delta u_t^T Q_{uu} \Delta u_t + \Delta u_t^TQ_{ux} \Delta x_t 
\end{equation}

where $\Delta u_t = u_t - u_t^r$, and $\Delta x_t = x_t - x_t^r$.

The first-order and second-order partial derivatives of $Q$ can be calculated as:

\begin{align}
Q_x &= \ell_x + f_x^TV_{x,t+1} \\
Q_u &= \ell_u + f_u^TV_{x,t+1} \\
Q_{xx} &= \ell_{xx} + f_x^TV_{xx,t+1} f_x + V_{x,t+1} f_{xx} \\
Q_{uu} &= \ell_{uu} + f_u^TV_{uu,t+1} f_u + V_{x,t+1} f_{uu} \\
Q_{ux} &= \ell_{ux} + f_u^TV_{ux,t+1} f_x + V_{x,t+1} f_{ux}
\end{align}


Then optimal control problem then becomes
\begin{equation}
    \Delta u^* = \argmin_{\Delta u} \hat Q(x, u) = -Q_{uu}^{-1}Q_u - Q_{uu}^{-1}Q_{ux}\Delta x = -k - K\Delta x
\end{equation}

Use this optimal $\Delta u$ and known nominal control inputs $u^r$, we can then calculate the optimal cost-to-go $V$ in a backward pass from $t = N$ to $t = 1$:

\begin{align}
u_t^* &= u_t^r + \Delta u_t^* \\
V(x_t) &= Q(x_t, u_t^*) = Q(x_t^r, u_t^r) - Q_uQ_{uu}^{-1}Q_u \\
V_x &= Q_x - Q_{ux}Q_{uu}^{-1}Q_u \\ 
V_{xx} &= Q_{xx} - Q_{ux}Q_{uu}^{-1}Q_{ux}
\end{align}

Then we can perform a forward pass from $t= 0$ to $t=N-1$ to calculate the new nominal state trajectories using the optimal control inputs:

\begin{align}
\hat x_0 &= x(0) \\
\Delta \hat u_t &= \hat u_t - u_t^r = -k_t - K_t(\hat x_t - x_t^r) \\
\hat x_{t+1} &= f(\hat x_t, \hat u_t)
\end{align}

The backward pass and forward pass are iterated until convergence.


**Procedure**
- generate a nominal trajectory for control inputs $\{u_0^r, u_1^r, ..., u_{N-1}^r\}$
- forward pass to calculate the nominal trajectory for system states $\{x_0^r, x_1^r, ..., x_N^r\}$
- set $V_N(x_N) = \ell_f(x_N)$, and calculate $V_x$ and $V_{xx}$ at $t = N$
- backward pass: for t = N, ..., 1:
  - calculate $Q_x$, $Q_u$, $Q_{xx}$, $Q_{uu}$ and $Q_{ux}$ at $t-1$
  - calculate $k_{t-1}, K_{t-1}$
- initiate $\hat x_0 = x(0)$
- forward pass: for t = 0, ..., N-1:
  - calculate $\Delta x_t = \hat x_t - x_t^r$
  - calculate control input changes $\Delta \hat u_t$ based on $k_t$, $K_t$ and $\Delta x_t$
  - calculate improved control input $\hat u_t = u_t^r + \Delta \hat u_t$
  - calculate improved state trajectory $\hat x_{t+1}$
- repeat until convergence
  - $V(x_0)$ is minimized and converged
  - max iteration is reached 
  - etc ... 
  
  


## Example

## Deveopment Ideas
- DDP class
- interact with openGym environment